In [1]:
import gym
import time
from tqdm import tqdm
import random
import os

import numpy as np
import time
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.distributions import MultivariateNormal

Fix environment

In [2]:
seed = 666 # Do not change this
device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
def fix(env, seed):
    env.seed(seed)
    env.action_space.seed(seed)
    T.manual_seed(seed)
    T.cuda.manual_seed(seed)
    T.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
  #torch.set_deterministic(True)
    T.backends.cudnn.benchmark = False
    T.backends.cudnn.deterministic = True

Construct the (linear) network structure--

In [3]:
class FeedForwardNN(nn.Module):
    def __init__(self, in_dim, out_dim, name, chkpt_dir):
        super(FeedForwardNN, self).__init__()
    
        self.chkpt_file = os.path.join(chkpt_dir, name)
    
        self.layer1 = nn.Linear(in_dim, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, out_dim)
    
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
    
        self.to(self.device)
    
    def forward(self, obs):
        x = F.relu(self.layer1(obs))
        x = F.relu(self.layer2(x))
        output = self.layer3(x)
        return output

    def save_checkpoint(self):
        T.save(self.state_dict(), self.chkpt_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.chkpt_file))

Create buffer

Construct our PPO class (which is also our agent) --

In [4]:
class PPO:
    def __init__(self, network, obs_dim, act_dim,lr = 0.005, gamma = 0.99, clip = 0.2, n_updates_per_iteration = 5, \
                max_timesteps_per_episode = 200, timesteps_per_batch = 2048, chkpt_dir = 'model/'):
        self.chkpt_dir = chkpt_dir
        
        self.timesteps_per_batch = timesteps_per_batch                 # Number of timesteps to run per batch
        self.max_timesteps_per_episode = max_timesteps_per_episode          # Max number of timesteps per episode
        self.n_updates_per_iteration = n_updates_per_iteration                # Number of times to update actor/critic per iteration
        self.lr = lr                                 # Learning rate of actor optimizer
        self.gamma = gamma                               # Discount factor to be applied when calculating Rewards-To-Go
        self.clip = clip                                 # Recommended 0.2, helps define the threshold to clip the ratio during SGA
        
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        
        self.actor = network(self.obs_dim, self.act_dim, 'ppo_actor.pth', self.chkpt_dir) 
        self.critic = network(self.obs_dim, 1, 'ppo_critic.pth', self.chkpt_dir)
        
        # Initialize optimizers for actor and critic
        self.actor_optim = Adam(self.actor.parameters(), lr=self.lr)
        self.critic_optim = Adam(self.critic.parameters(), lr=self.lr)
        
        # Initialize the covariance matrix used to query the actor for actions
        self.cov_var = T.full(size=(self.act_dim,), fill_value=0.5)
        self.cov_mat = T.diag(self.cov_var)
        
    def learn(self, batch_obs, batch_acts, batch_rews, batch_log_probs):
        
        device = self.actor.device
        
        batch_rtgs = self.compute_rtgs(batch_rews)
        batch_rtgs = T.tensor(np.array(batch_rtgs), dtype=T.float).to(device)
        batch_obs = T.tensor(np.array(batch_obs), dtype=T.float).to(device)
        batch_acts = T.tensor(np.array(batch_acts), dtype=T.float).to(device)
        batch_log_probs = T.tensor(np.array(batch_log_probs), dtype=T.float).to(device)
        
        # Calculate advantage at k-th iteration
        V, _ = self.evaluate(batch_obs, batch_acts)
        A_k = batch_rtgs - V.detach()          
        A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10) # Normalizing the advantage
        
        for _ in range(self.n_updates_per_iteration):
            V, curr_log_probs = self.evaluate(batch_obs, batch_acts)
            
            ratios = T.exp(curr_log_probs - batch_log_probs)
            
            # Calculate surrogate losses.
            surr1 = ratios * A_k
            surr2 = T.clamp(ratios, 1 - self.clip, 1 + self.clip) * A_k
            
            actor_loss = (-T.min(surr1, surr2)).mean()
            critic_loss = nn.MSELoss()(V, batch_rtgs)
            
            # Calculate gradients and perform backward propagation for actor network
            self.actor_optim.zero_grad()
            actor_loss.backward(retain_graph=True)
            self.actor_optim.step()
            
            # Calculate gradients and perform backward propagation for critic network
            self.critic_optim.zero_grad()
            critic_loss.backward()
            self.critic_optim.step()    
            
    def compute_rtgs(self, batch_rews):
        batch_rtgs = []
        
        for epoch_rews in reversed(batch_rews):
            discounted_reward = 0
            for rew in reversed(epoch_rews):
                discounted_reward = rew + discounted_reward * self.gamma
                batch_rtgs.insert(0, discounted_reward)
                
        
        return batch_rtgs
        
    def get_action(self, obs):
        state = T.tensor(obs, dtype=T.float).to(self.actor.device)
        mean = self.actor.forward(state)
        
        distr = MultivariateNormal(mean, self.cov_mat)
        action = distr.sample()
        log_prob = distr.log_prob(action)
        return action.detach().cpu().numpy(), log_prob.detach().cpu().numpy()
    
        
    def evaluate(self, batch_obs, batch_acts):
        # Estimate the values of each observation, and the log probs of
        # each action in the most recent batch with the most recent
        # iteration of the actor network. Should be called from learn.
        
        # Query critic network for a value V for each batch_obs. Shape of V should be same as batch_rtgs
        V = self.critic.forward(batch_obs).squeeze()
        
        # Calculate the log probabilities of batch actions using most recent actor network.
        mean = self.actor.forward(batch_obs)
        dist = MultivariateNormal(mean, self.cov_mat)
        log_probs = dist.log_prob(batch_acts)

        # V - the predicted values of batch_obs
        # log_probs - the log probabilities of the actions taken in batch_acts given batch_obs
        return V, log_probs
    
        

Define the function to collect the on-policy trajectories--

In [5]:
def rollout(policy, env, timesteps_per_batch, max_timesteps_per_episode):
    # Batch data.
    batch_obs = []
    batch_acts = []
    batch_log_probs = []
    batch_rews = []
    batch_lens = []
    
    epoch_rews = []
    t=0
    while t < timesteps_per_batch:
        epoch_rews = [] # rewards collected per episode
        obs = env.reset()
        done = False
        for h in range(max_timesteps_per_episode):
            batch_obs.append(obs)
            action, log_prob = policy.get_action(obs)
            obs, rew, done, _ = env.step(action)
            t +=1
            
            epoch_rews.append(rew)
            batch_acts.append(action)
            batch_log_probs.append(log_prob)
            
            if done:
                break
            
        batch_lens.append(h+1)
        batch_rews.append(epoch_rews)
    
    
    
    return batch_obs, batch_acts, batch_rews, batch_log_probs, batch_lens

In [6]:
hyperparameters = {
    'timesteps_per_batch': 2048, 
    'max_timesteps_per_episode': 200, 
    'gamma': 0.99, 
    'n_updates_per_iteration': 10,
    'lr': 3e-4, 
    'clip': 0.2,
    'seed': 666
    }

The training loop----

In [7]:
env = gym.make('Pendulum-v0')
fix(env, seed)
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

model = PPO(network=FeedForwardNN, obs_dim = obs_dim, act_dim = act_dim)
model.actor.load_checkpoint()
model.critic.load_checkpoint()
model.actor.train()
model.critic.train()


FeedForwardNN(
  (layer1): Linear(in_features=3, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=1, bias=True)
)

Train--

In [8]:
ITERATIONS = 100
tmp = 0.0

prg_bar = tqdm(range(ITERATIONS))
for i in prg_bar:
    batch_obs, batch_acts, batch_rews, batch_log_probs, batch_lens = \
        rollout(model, env, hyperparameters['timesteps_per_batch'], hyperparameters['max_timesteps_per_episode'])

    avg_ep_lens = np.mean(batch_lens)
    avg_ep_rews = np.mean([np.sum(ep_rews) for ep_rews in batch_rews])
    
    # Round decimal places for more aesthetic logging messages
    avg_ep_lens = str(round(avg_ep_lens, 2))
    avg_ep_rews_1 = str(round(avg_ep_rews, 2))

    print(f"-------------------- Iteration --------------------", flush=True)
    print(f"Average Episodic Length: {avg_ep_lens}", flush=True)
    print(f"Average Episodic Return: {avg_ep_rews_1}", flush=True)
    
    model.learn(batch_obs, batch_acts, batch_rews, batch_log_probs)
    if avg_ep_rews > tmp:
        model.actor.save_checkpoint()
        model.critic.save_checkpoint()
        print("Successfully save the model!",flush=True)
    tmp = avg_ep_rews
    

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


-------------------- Iteration --------------------
Average Episodic Length: 200.0
Average Episodic Return: -150.86


  1%|▊                                                                                 | 1/100 [00:01<02:58,  1.81s/it]


-------------------- Iteration --------------------
Average Episodic Length: 200.0
Average Episodic Return: -181.43


  2%|█▋                                                                                | 2/100 [00:03<02:56,  1.80s/it]

...


-------------------- Iteration --------------------
Average Episodic Length: 200.0
Average Episodic Return: -167.67


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [03:10<00:01,  1.91s/it]

Successfully save the model!

-------------------- Iteration --------------------
Average Episodic Length: 200.0
Average Episodic Return: -154.52


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:12<00:00,  1.92s/it]

Successfully save the model!


Test----

In [9]:
env = gym.make('Pendulum-v0')
fix(env, seed)
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

model = PPO(network=FeedForwardNN, obs_dim = obs_dim, act_dim = act_dim)
model.actor.load_checkpoint()
model.critic.load_checkpoint()
model.actor.eval()  # turn network to evaluation mode
model.critic.eval()
NUM_OF_TEST = 5 # Do not revise it !!!!!
test_total_reward = []
action_list = []
for i in range(NUM_OF_TEST):
    actions = []
    obs = env.reset()

  #img = plt.imshow(env.render(mode='rgb_array'))
    env.render()
  
    total_reward = 0

    done = False
    while not done:
        action, _ = model.get_action(obs)
        actions.append(action)
        obs, reward, done, _ = env.step(action)

        total_reward += reward

      #img.set_data(env.render(mode='rgb_array'))
        env.render()
      #display.display(plt.gcf())
      #display.clear_output(wait=True)
    print(total_reward)
    test_total_reward.append(total_reward)


env.close()

print(f"Your final reward is : %.2f"%np.mean(test_total_reward))

-120.21181737412071
-125.41116676883169
-243.96041123861386
-338.05871844153194
-242.14872698933092
Your final reward is : -213.96
